In [1]:
!pip install datasets


In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("shawhin/lead-scoring-x")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/568 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2561 [00:00<?, ? examples/s]

In [3]:
df = pd.DataFrame(dataset['train'])
df.head()

,Prospect ID,Lead Origin,Lead Source,Last Activity,Tags,What is your current occupation,Converted
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,API,Olark Chat,Page Visited on Website,Interested in other courses,Unemployed,0
1,2a272436-5132-4136-86fa-dcc88c88f482,API,Organic Search,Email Opened,Ringing,Unemployed,0
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,Landing Page Submission,Direct Traffic,Email Opened,Will revert after reading the email,Student,1
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,Landing Page Submission,Direct Traffic,Unreachable,Ringing,Unemployed,0
4,3256f628-e534-4826-9d63-4a8b88782852,Landing Page Submission,Google,Converted to Lead,Will revert after reading the email,Unemployed,1


In [4]:
df.columns

Index(['Prospect ID', 'Lead Origin', 'Lead Source', 'Last Activity', 'Tags',
       'What is your current occupation', 'Converted'],
      dtype='object')

In [5]:
df['Converted'].value_counts()

,count
Converted,
0,312
1,256


In [6]:
df = df.fillna("Unknown")

In [7]:
df_encoded = pd.get_dummies(df, drop_first=True)

In [8]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('Converted', axis=1)
y = df_encoded['Converted']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [10]:
lead_probabilities = model.predict_proba(X)[:, 1]

In [11]:
def lead_category(p):
    if p >= 0.7:
        return "High"
    elif p >= 0.4:
        return "Medium"
    else:
        return "Low"

df['lead_score'] = lead_probabilities
df['lead_category'] = df['lead_score'].apply(lead_category)

In [12]:
final_output = df[['lead_score', 'lead_category']]
final_output.to_csv("lead_scores.csv", index=False)

final_output.head()

,lead_score,lead_category
0,0.056263,Low
1,0.024487,Low
2,0.980333,High
3,0.030681,Low
4,0.958075,High
